# Pruning Algorithm
Our **MiniMax Algorithm** is superior when the given game is:
* Small
* Deterministic
* Has Perfect Information 

For example: A Game of Tic-Tac-Toe would be the perfect 'enviroment' whereas there are ~300,000 possible game scenarios. 

This may become troublesome when building algorithms for much more complex games such as: Chess, Checkers, Go. In fact in Chess & Go, there are more possible board positions than there are atoms in the obersvable universe. 

This is where **Pruning** comes into play.

## Trees
We need to think of these algorithms as trees where the **Width** of the tree represents the number of possible moves and the **Depth** represents the number of turns. 

So a thought experiment to have before building any algorithm for a game is to: *Estimate the **size** of the tree.* 

A good way to estimate this is: 
$$W^d$$

* $W$ = Average width
* $d$ = Average depth

Example: 

**Go** typically has **250** legal moves **per turn**, and a game might last **150 turns**. 
Thus: 

$$W^d ≈ 250^{150} ≈ 10^{359}$$

That is, $10^{359}$ positions. That's 359 zeros... 

## Pruning
The intuition behind **Pruning** is that when reducing the width or depth of our (tree) search, we drastically reduce the time needed to select a move.

### Position Evaluation Functions
Are used for reducing the **Search Depth**. The intuition behind this is the sense of who's in the lead.

In [4]:
def best_result(game_state, max_depth, eval_fn):
    """
    This function will return a number indicating the value of your board evaluation function. A large score means the player who has the next move expects to win. When you evaluate the board from your opponents perspective, you multiply the score by -1 to flip back to your perspective. 
    
    ARGs:
        game_state: the given game state of that legal move (usualy future game state)
        max_depth: controls the number of moves you want to search ahead. We subtract 1 each turn, then call our board evaluation function
        eval_fn: A functions who's job is to evaluate the board 
    """
    # Checking if game is over, this is a recursive function so we call this first
    if game_state.is_over():
        if game_state.winner() == game_state.next_player:
            return MAX_SCORE # 3
        else:
            return MIN_SCORE # 1
    
    # Have we reached the maximum depth search? - we subtract one every time this function is called 
    if max_depth == 0:
        return eval_fn(game_state) # this is essentually the 'loss function'
    
    # Initialize with lowest score
    best_so_far = MIN_SCORE
    
    # Loop through all possible, legal moves
    for candidate_move in game_state.legal_moves():
        
        # Calculating what the board would look like playing this legal move
        next_state = game_state.apply_move(candidate_move)
        
        # What is the opponents best position if we play that legal move?
        opponent_best_result = best_result(next_state, max_depth - 1, eval_fn)
        
        # Inver their best results
        our_result = -1 * opponent_best_result
        
        # is this better than our best result so far?
        if our_result > best_result:
            best_so_far = our_result
            
    return best_so_far

In [5]:
# EXAMPLE EVAL FUNCTION - highly simplified board evaluation heuristic for Go. Check out: www.github.com/dmbernaal/SigmaGo.git for details on other variables & functions here 
def capture_diff(game_state):
    """
    This is a simple board evaluation heuristic for go. Such evaluation function would go as the second argument in best_result()
    
    This function calculates the difference in # of stones captured. The player with more stones captured is the 'leader' 
    """
    black_stones = 0 
    white_stones = 0
    for r in range(1, game_state.board.num_rows + 1):
        for c in range(1, game_state.board.num_cols + 1):
            p = gotypes.Point(r, c)
            color = game_state.board.get(p)
            if color == gotypes.Player.black:
                black_stones += 1
            elif color == gotypes.Player.white:
                white_stones += 1
    diff = black_stones - white_stones
    if game_state.next_player == gotypes.Player.black:
        return diff
    return -1 * diff # if player is white

### Alpha-Beta Pruning
Are used for reducing the **Search Width** The time savings you get by alpha-beta pruning depends on how quickly you find good branches. If you happen to evaluate the best branches early on, you can eliminate the other branches quickly. 

To implement this algorithm, you must track the best result so far for each player throughout your search:
* **Alpha** = ```best_black```
* **Beta** = ```best_white```

In [ ]:
def alpha_beta_result(game_state, max_depth, best_black, best_white, eval_fn):
    """
    ARGs:
        game_state: The given game state provided the move
        max_depth: How deep in the tree will we evaluate? - similar to other function
        best_black: alpha or beta?
        best_white: alpha or beta?  (opposite of best_black)
        eval_fn: our 'loss function', we use capture_diff() in this example
    """
    # Checking if game is already over
    if game_state.is_over():
        if game_state.winner() == game_state.next_player:
            return MAX_SCORE 
        else:
            return MIN_SCORE
     
    # have you reached your maximum depth?
    if max_depth == 0:
        return eval_fn(game_state)
    
    # Initializing best so far = 1
    best_so_far = MIN_SCORE
    
    # Looping over all legal moves
    for candidate_move in game_state.legal_moves():
        
        # What will the board look like if you play this legal move?
        next_state = game_state.apply_move(candidate_move)
        
        # What is your opponents best result from that position? (move)
        opponent_best_result = alpha_beta_result(next_state, max_depth - 1, best_black, best_white, eval_fn)
        
        # Whatever our opponent wants, we want the opposite
        our_result = -1 * opponent_best_result
        
        # Is our result the best we've seen so far? 
        if our_result > best_so_far:
            best_so_far = our_result
        
        # updating benchmark for white
        if game_state.next_player == Player.white:
            if best_so_far > best_white:
                best_white = best_so_far
            outcome_for_black = -1 * best_so_far
            
            # picking the best move for white
            if outcome_for_black < best_black:
                return best_so_far
        
        # updating benchmark for black
        elif game_state.next_player == Player.black:
            if best_so_far > best_black:
                best_black = best_so_far
            outcome_for_white = -1 * best_so_far
            
            # picking best move for black
            if outcome_for_white < best_white:
                return best_so_far
    
    return best_so_far